In [ ]:
#preprocessing of the dataset: - resize the image to 512x512 - denoise the image - normalize the image
import cv2

def preprocess_image(image):
    # Resize image to desired dimensions
    resized_image = cv2.resize(image, (512, 512))

    # Apply denoising and normalization techniques
    denoised_image = cv2.fastNlMeansDenoising(resized_image)
    normalized_image = cv2.normalize(denoised_image, None, 0, 255, cv2.NORM_MINMAX)

    return normalized_image

# Load and preprocess the CT scan image
image = cv2.imread('image_path', cv2.IMREAD_GRAYSCALE)
preprocessed_image = preprocess_image(image)


In [ ]:
#bone identification: - apply a threshold to the image - apply a morphological operation to remove noise
import numpy as np

def segment_bones(image):
    # Apply a thresholding technique to segment bone regions
    _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

    # Further refine the segmentation using morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    segmented_image = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel)

    return segmented_image

# Segment bones from the preprocessed image
bone_segmentation = segment_bones(preprocessed_image)


In [ ]:
#localization of vertebral columns: - apply a threshold to the image - apply a morphological operation to remove noise
from skimage import measure

def localize_columns(segmented_image):
    # Apply connected component analysis to extract individual bone components
    labels = measure.label(segmented_image, connectivity=2)

    # Identify and localize the vertebral columns
    columns = []
    for label in np.unique(labels):
        if label == 0:
            continue
        mask = np.where(labels == label, 1, 0)
        column_bounding_box = np.array([np.min(np.nonzero(mask)[0]), np.max(np.nonzero(mask)[0]),
                                        np.min(np.nonzero(mask)[1]), np.max(np.nonzero(mask)[1])])
        columns.append(column_bounding_box)

    return columns

# Localize vertebral columns from the bone segmentation
column_localizations = localize_columns(bone_segmentation)


In [ ]:
#segmentation of vertebra: - apply a threshold to the image - apply a morphological operation to remove noise
from skimage.measure import label
from skimage.segmentation import flood

def segment_vertebrae(column_image):
    # Apply region growing algorithm to segment vertebrae within a column
    labeled_image = label(column_image)
    num_vertebrae = np.max(labeled_image)

    # Extract individual vertebrae using labeled regions
    vertebrae_masks = []
    for i in range(1, num_vertebrae + 1):
        vertebra_mask = np.where(labeled_image == i, 1, 0)

        # Perform region growing starting from a seed point
        seed_point = find_seed_point(vertebra_mask)  # You need to implement this function
        segmented_vertebra = flood(column_image, seed_point)

        vertebrae_masks.append(segmented_vertebra)

    return vertebrae_masks

# Localize vertebral columns from the bone segmentation
column_localizations = localize_columns(bone_segmentation)

# Segment vertebrae within each vertebral column
vertebrae_segmentation = []
for column in column_localizations:
    column_image = bone_segmentation[column[0]:column[1], column[2]:column[3]]
    vertebrae_masks = segment_vertebrae(column_image)
    vertebrae_segmentation.append(vertebrae_masks)

